In [24]:
import numpy as np, pandas as pd, matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
import time

In [25]:
df = pd.read_csv(r"Dataset/Car Sell Dataset.csv")

In [26]:
df.head()

,Brand,Model Name,Model Variant,Car Type,Transmission,Fuel Type,Year,Kilometers,Owner,State,Accidental,Price
0,Mahindra,TUV300,AX5,SUV,Manual,CNG,2017,164654,1st,Rajasthan,No,547253
1,Skoda,Rapid,Style,Sedan,Manual,Petrol,2018,41351,1st,Maharashtra,No,512050
2,Maruti Suzuki,Alto,Z,Hatchback,Manual,Diesel,2002,119090,3rd+,Tamil Nadu,No,678923
3,Hyundai,Grand i10,Magna,Hatchback,Manual,Diesel,2013,19979,1st,Andhra Pradesh,No,522500
4,Mahindra,XUV500,W8,SUV,Manual,Petrol,2011,130591,3rd+,Bihar,No,401182


In [27]:
fuel_dict = {
  'Petrol':1,
  'Diesel':2,
  'CNG':3,
  'Electric':4,
  'Hybrid':5
}

In [28]:
df['Fuel Type'] = df['Fuel Type'].map(fuel_dict)
df.head()

,Brand,Model Name,Model Variant,Car Type,Transmission,Fuel Type,Year,Kilometers,Owner,State,Accidental,Price
0,Mahindra,TUV300,AX5,SUV,Manual,3,2017,164654,1st,Rajasthan,No,547253
1,Skoda,Rapid,Style,Sedan,Manual,1,2018,41351,1st,Maharashtra,No,512050
2,Maruti Suzuki,Alto,Z,Hatchback,Manual,2,2002,119090,3rd+,Tamil Nadu,No,678923
3,Hyundai,Grand i10,Magna,Hatchback,Manual,2,2013,19979,1st,Andhra Pradesh,No,522500
4,Mahindra,XUV500,W8,SUV,Manual,1,2011,130591,3rd+,Bihar,No,401182


In [30]:
x11_np = np.array(df['Fuel Type'].values)
x21_np = np.array(df['Kilometers'].values)
y_np = np.array(df['Price'].values)

x11_np = (x11_np - x11_np.mean()) / x11_np.std()
x21_np = (x21_np - x21_np.mean()) / x21_np.std()
y_np   = (y_np   - y_np.mean())  / y_np.std()

device = torch.device('cuda')

x11 = torch.tensor(x11_np, device=device)
x21 = torch.tensor(x21_np, device=device)
y = torch.tensor(y_np, device=device)

In [38]:
w0 = torch.rand(1, requires_grad=True, dtype=torch.float64, device=device)
w1 = torch.rand(1, requires_grad=True, dtype=torch.float64, device=device)
w2 = torch.rand(1, requires_grad=True, dtype=torch.float64, device=device)

In [39]:
epoch_num = 10
learning_rate = 0.001


for epoch in range(epoch_num):
    epoch_loss=0
    for i in range(len(x11)):
        z = w0 + (w1*x11[i]) + (w2*x21[i])
        y_hat = torch.sigmoid(z)
        #Calculating loss
        loss_tensor = torch.pow(y_hat-y[i],2)
        epoch_loss += loss_tensor.item()

        loss_tensor.backward()

        with torch.no_grad():
            w0 -= w0.grad * learning_rate
            w1 -= w1.grad * learning_rate
            w2 -= w2.grad * learning_rate

        w0.grad.zero_()
        w1.grad.zero_()
        w2.grad.zero_()    
    
    print(f"Epoch {epoch+1}, Loss: {epoch_loss/len(x11)}")

Epoch 1, Loss: 0.9955943316657907
Epoch 2, Loss: 0.9731210130764707
Epoch 3, Loss: 0.9722473925894458
Epoch 4, Loss: 0.9719854055071945
Epoch 5, Loss: 0.971882452373706
Epoch 6, Loss: 0.9718365288052824
Epoch 7, Loss: 0.9718145076040136
Epoch 8, Loss: 0.971803457601292
Epoch 9, Loss: 0.9717977432525712
Epoch 10, Loss: 0.9717947263113359


In [33]:
torch.cuda.is_available()

True

In [35]:
#Tensors on CPU

tensor1 = torch.rand(10000,10000)
tensor2 = torch.rand(10000,10000)

cpu_start_time = time.time()
tensor11 = torch.matmul(tensor1, tensor2)
cpu_time_taken = time.time() - cpu_start_time

print(f"Time taken by CPU: {cpu_time_taken}")

#Tensors on GPU
tensor1_gpu = torch.rand(10000,10000,device=device)
tensor2_gpu = torch.rand(10000,10000,device=device)

gpu_start_time = time.time()
tensor11_gpu = torch.matmul(tensor1_gpu, tensor2_gpu)
torch.cuda.synchronize()
gpu_time_taken = time.time() - gpu_start_time

print(f"Time taken by GPU: {gpu_time_taken}")

Time taken by CPU: 4.185971260070801
Time taken by GPU: 0.2926809787750244
